### IMPORT LIBRARIES & CREATE CONNECTOR

In [17]:
import pandas as pd
import numpy as np
import sqlalchemy as sa
import mysql.connector
import getpass

In [18]:
HOST = 'localhost'
USER = 'root'
DATABASE = 'goodreads'
PASSWORD = getpass.getpass(f'Enter password for {USER}: ')

In [19]:
def getconn():
    conn = mysql.connector.connect(
        host=HOST,
        user=USER,
        password=PASSWORD,
        database=DATABASE
    )
    return conn

pool = sa.create_engine(
    "mysql+mysqlconnector://",
    creator=getconn,
)

with pool.connect() as db_conn:
    results = db_conn.execute(sa.text("SELECT NOW()")).fetchone()
    print("Current time: ", results[0])

Current time:  2023-05-24 14:34:22


### SANDBOX

In [20]:
book_id_query = sa.text(
    "SELECT goodreads_book_id FROM books;"
)
book_id = pd.read_sql_query(book_id_query, con=pool.connect())
book_id.head()

,goodreads_book_id
0,2767052
1,3
2,41865
3,2657
4,4671


In [21]:
id_list = book_id.goodreads_book_id.tolist()

In [22]:
user_id_query = sa.text(
    "SELECT DISTINCT(user_id) FROM new_ratings;"
)
user_id = pd.read_sql_query(user_id_query, con=pool.connect())
user_id.head()

,user_id
0,1
1,2
2,4
3,8
4,9


In [23]:
user_list = user_id.user_id.tolist()
user_list[:5]

['1', '2', '4', '8', '9']

In [24]:
columns = list(['user_id'] + id_list)

In [25]:
user_ratings_df = pd.DataFrame(columns=columns)
user_ratings_df

,user_id,2767052,3,41865,2657,4671,11870085,5907,5107,960,...,101094,13616278,4936457,4769651,15613,7130616,208324,77431,8565083,8914


In [26]:
def get_ratings(user_id, book_id):
    query = sa.text(
        f"SELECT rating FROM new_ratings WHERE user_id = {user_id} AND goodreads_book_id = {book_id};"
    )
    result = pd.read_sql_query(query, con=pool.connect())
    return result

In [27]:
sample = get_ratings('194', '374233')
sample

,rating
0,4
1,4
2,4
